In [3]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
home_dir = "../../"

import time
import os
import pandas as pd

In [4]:
gene2refseq_mapping_filepath = home_dir+"data/refseq/MANE.GRCh38.v1.0.summary.txt.gz" # "symbol" is the Gene-symbol col
# gene2refseq_mapping_filepath = home_dir+"data/gene/gene2refseq_filtered.gz" # "Symbol" is the Gene-symbol col
# gene2refseq_mapping_filepath = home_dir+"data/refseq/HumanLRGRefSeqGene.txt.gz"  # "Symbol" is the Gene-symbol col

gene_symbol_col_name = "symbol" if gene2refseq_mapping_filepath.__contains__("MANE") else "Symbol"
gene2refseq_mapping_df = pd.read_csv(gene2refseq_mapping_filepath, compression='gzip', delim_whitespace=False, sep="\t")#, header=None, names=col_names)

print(gene2refseq_mapping_df.shape)
print(len(gene2refseq_mapping_df[gene_symbol_col_name].unique())) # some gene can be mapped to multiple proteins
gene2refseq_mapping_df.head()

(19120, 14)
19062


,#NCBI_GeneID,Ensembl_Gene,HGNC_ID,symbol,name,RefSeq_nuc,RefSeq_prot,Ensembl_nuc,Ensembl_prot,MANE_status,GRCh38_chr,chr_start,chr_end,chr_strand
0,GeneID:1,ENSG00000121410.12,HGNC:5,A1BG,alpha-1-B glycoprotein,NM_130786.4,NP_570602.2,ENST00000263100.8,ENSP00000263100.2,MANE Select,19,58345183,58353492,-
1,GeneID:2,ENSG00000175899.15,HGNC:7,A2M,alpha-2-macroglobulin,NM_000014.6,NP_000005.3,ENST00000318602.12,ENSP00000323929.8,MANE Select,12,9067708,9115919,-
2,GeneID:9,ENSG00000171428.15,HGNC:7645,NAT1,N-acetyltransferase 1,NM_000662.8,NP_000653.3,ENST00000307719.9,ENSP00000307218.4,MANE Select,8,18210109,18223689,+
3,GeneID:10,ENSG00000156006.5,HGNC:7646,NAT2,N-acetyltransferase 2,NM_000015.3,NP_000006.2,ENST00000286479.4,ENSP00000286479.3,MANE Select,8,18391282,18401218,+
4,GeneID:12,ENSG00000196136.18,HGNC:16,SERPINA3,serpin family A member 3,NM_001085.5,NP_001076.2,ENST00000393078.5,ENSP00000376793.3,MANE Select,14,94612391,94624053,+


In [29]:
from data_loader import get_population_freq_SNVs
variants_df = get_population_freq_SNVs(home_dir=home_dir)
merged_df = pd.merge(variants_df, gene2refseq_mapping_df, how="inner", left_on="prot_acc_version", right_on="RefSeq_prot")
columns = list(variants_df.columns)
columns.append("RefSeq_nuc")
merged_df = merged_df[columns]
merged_df = merged_df.rename(columns={"RefSeq_nuc":"nuc_acc_version"})
# merged_df["RefSeq_nuc"].value_counts()
# merged_df["RefSeq_nuc"].unique().tolist()
merged_df.to_csv(home_dir+"models/aa_common/datasets_population_freq/SNVs_with_popu_freq_balanced_with_NMs.txt", sep="\t", header=True, index=False)
merged_df


Log: Loading data ...
(95223, 13)
Index(['snp_id', 'chrom_acc_version', 'chrom_pos', 'ref_allele', 'alt_allele',
       'prot_acc_version', 'prot_pos', 'wt', 'mut', 'wt_population',
       'mut_poulation', 'wt_freq', 'mt_freq'],
      dtype='object')
After combining common (18279), rare (29383) and sampled-singletons (48434), data: (95223, 13)
NP_001277137.1    95
NP_001291317.1    78
NP_002115.2       74
NP_112241.2       68
NP_001075106.2    65
                  ..
NP_037408.2        1
NP_066951.1        1
NP_001013716.2     1
NP_001316544.1     1
NP_065171.2        1
Name: prot_acc_version, Length: 14997, dtype: int64


,snp_id,chrom_acc_version,chrom_pos,ref_allele,alt_allele,prot_acc_version,prot_pos,wt,mut,wt_population,mut_poulation,wt_freq,mt_freq,nuc_acc_version
0,rs41288789,NC_000002.12,26944749,G,A,NP_064519.2,512,A,T,218102,10302,0.954896,0.045104,NM_020134.4
1,rs151073506,NC_000002.12,26934674,C,T,NP_064519.2,296,T,M,203738,622,0.996956,0.003044,NM_020134.4
2,rs752500593,NC_000002.12,26928295,A,G,NP_064519.2,214,E,G,23038,1,0.999957,0.000043,NM_020134.4
3,rs1229783597,NC_000002.12,26924896,G,A,NP_064519.2,91,V,I,23038,1,0.999957,0.000043,NM_020134.4
4,rs55675629,NC_000002.12,26944704,G,A,NP_064519.2,497,A,T,35432,1,0.999972,0.000028,NM_020134.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95218,rs781903679,NC_000023.11,73079227,C,T,NP_001012995.1,54,V,M,17528,1,0.999943,0.000057,NM_001012977.3
95219,rs145143247,NC_000007.14,107595887,C,T,NP_061332.2,122,T,M,14050,1,0.999929,0.000071,NM_018844.4
95220,rs1249874393,NC_000012.12,49067035,G,C,NP_653194.1,42,T,I,35232,1,0.999972,0.000028,NM_144593.3
95221,rs1364156566,NC_000006.12,27893067,A,G,NP_003505.1,28,V,A,19668,1,0.999949,0.000051,NM_003514.2
